# Landscape Ecology

__Evelyn Uuemaa, PhD__

- Course supplement

    Students upload a zip file with landscape rasters (all the same format, e.g. *.tif) and calculate selected landscape metrics. They are then also able to download the statistics for reporting.

References:
- https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0225734
- https://github.com/martibosch/pylandstats

## How to use this notebook

Highlight the top cell and click the "Run" button to execute the cells and move the program forward. If anything goes wrong, you can easily restart from the beginning.

In [1]:
from IPython.display import display
import ipywidgets as widgets
import zipfile
import glob
from tqdm import tqdm 

In [2]:
upload = widgets.FileUpload( accept='.zip', multiple=False )

In [3]:
upload

FileUpload(value={}, accept='.zip', description='Upload')

After you selected the file, the upload field should show (1), that means the file is now selected.

Click on the _next cell_ to continue...

In [4]:
t = upload.data[0]

with open('tmp.zip', 'wb') as fh:
    fh.write(t)


with zipfile.ZipFile('tmp.zip', 'r') as zip_ref:
    zip_ref.extractall(".")

all_tifs = glob.glob("*.tif")

# uncomment to see the names of all *.tif files inside the uploaded zip 
# print(all_tifs)

Now we have the data loaded and we can run the statistics.

In [5]:
import pandas as pd
import pylandstats as pls

all_metrics = []

for landscape in tqdm(all_tifs):
    ls = pls.Landscape(landscape)
    metrics_df = ls.compute_landscape_metrics_df(metrics=["edge_density", "patch_density", "shannon_diversity_index"])
    metrics_df["mean_shape_index"] = ls.shape_index()['shape_index'].mean()
    metrics_df["mean_patch_size_ha"] = ls.area(class_val=None, hectares=True)['area'].mean()
    metrics_df.index = [landscape]
    all_metrics.append(metrics_df)
    
all_metrics_df = pd.concat(all_metrics, axis=0)

all_metrics_df.to_csv("landscape_metrics.csv", sep=",", encoding="utf-8")

display(all_metrics_df)

100%|██████████| 2/2 [00:00<00:00, 24.78it/s]


,edge_density,patch_density,shannon_diversity_index,mean_shape_index,mean_patch_size_ha
36871.tif,41.554322,3.489294,1.785777,1.529613,28.659091
36870.tif,41.554322,3.489294,1.785777,1.529613,28.659091


[download the table here](landscape_metrics.csv)

Alternatively, just load the file directly from the file list